In [59]:
import pandas as pd
import numpy as np

In [60]:
import geopandas as gpd
import libpysal as lp

map_2023_path = "../../housing-data/cbsgebiedsindelingen2016-2025/cbsgebiedsindelingen2023.gpkg"


def read_file(file_path, layer="buurt_gegeneraliseerd"):
    gdf = gpd.read_file(file_path, layer=layer)
    #gdf = gdf[gdf['indelingswijziging_wijken_en_buurten'] > 0] -- Wel of niet???
    gdf_neighbors = lp.weights.Queen.from_dataframe(gdf, use_index=False)
    gdf_neighbors.to_sparse()
    codes = gdf.iloc[:, :1].to_numpy().flatten()
    adj_df = pd.DataFrame.sparse.from_spmatrix(
        gdf_neighbors.to_sparse(), index=codes, columns=codes
    )
    return adj_df, gdf

adj_df_2023, gdf_2023 = read_file(map_2023_path)

c:\Users\AratrikaD\rdlabs-gnns-for-property-valuation\.venv\lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 10 disconnected components.
 There are 4 islands with ids: 3025, 10085, 10252, 10810.
  W.__init__(self, neighbors, ids=ids, **kw)


In [61]:
df_synth = pd.read_parquet("../../housing-data/df_synth.parquet")
df_synth

,SELLING_PRICE,DATE_OF_SALE,HOUSETYPE,SOC_CODE,LIVING_AREA,PLOT_AREA,CONSTRUCTION_YEAR,LAT,LON,DESTINATION,STORAGE_OR_SHED,GARAGE,MONUMENT
0,410399.0,2023-11-23,0,22,69.0,50.0,2006,51.930438,4.597374,0,1,1,0
1,553211.0,2024-08-31,3,10,143.0,170.0,1964,51.914836,4.439337,0,1,0,0
2,281874.0,2024-03-15,0,22,103.0,53.0,1949,51.910093,4.372199,0,1,0,0
3,311783.0,2017-10-30,2,9,182.0,164.0,1969,51.918368,4.322173,0,0,0,0
4,864350.0,2023-08-03,3,10,148.0,322.0,1938,51.932097,4.464516,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,219846.0,2019-08-19,0,22,88.0,53.0,1951,51.910131,4.404810,0,1,0,0
199996,369542.0,2023-09-30,0,24,94.0,50.0,1970,51.921255,4.562029,0,1,0,0
199997,271956.0,2018-12-23,0,25,83.0,52.0,1924,51.907061,4.498734,0,1,0,0
199998,140154.0,2018-01-27,3,10,89.0,201.0,1980,51.936764,4.371009,0,1,0,0


In [62]:
gdf_2023

,statcode,jrstatcode,statnaam,gm_code,rubriek,id,geometry
0,BU00340101,2023BU00340101,Centrum Haven Noord,GM0034,buurt,1,"MULTIPOLYGON (((143508.204 483344.282, 143707...."
1,BU00340102,2023BU00340102,Centrum Haven Zuid,GM0034,buurt,2,"MULTIPOLYGON (((143728.65 483029.012, 143730.8..."
2,BU00340201,2023BU00340201,Rozenwerf en Tuinderswerf,GM0034,buurt,3,"MULTIPOLYGON (((144118.02 483591.859, 144295.6..."
3,BU00340202,2023BU00340202,Achterwerf,GM0034,buurt,4,"MULTIPOLYGON (((144427.109 483263.953, 144384...."
4,BU00340203,2023BU00340203,Goedewerf en Wittewerf,GM0034,buurt,5,"MULTIPOLYGON (((144267.821 483312.403, 144090...."
...,...,...,...,...,...,...,...
14416,BU03850501,2023BU03850501,Warder,GM0385,buurt,14417,"MULTIPOLYGON (((130959.382 509613.307, 130850...."
14417,BU03850601,2023BU03850601,Middelie,GM0385,buurt,14418,"MULTIPOLYGON (((128874.917 507747.278, 130244...."
14418,BU03850701,2023BU03850701,Kwadijk,GM0385,buurt,14419,"MULTIPOLYGON (((127500.256 506394.557, 128625...."
14419,BU03850801,2023BU03850801,Schardam,GM0385,buurt,14420,"MULTIPOLYGON (((130104.529 513486.749, 130091...."


In [63]:
from shapely.geometry import Point

# Convert transaction lat/lon to Point geometry
geometry = [Point(xy) for xy in zip(df_synth["LON"], df_synth["LAT"])]
transactions_gdf = gpd.GeoDataFrame(df_synth, geometry=geometry, crs="EPSG:4326")

# Ensure both datasets use the same coordinate reference system (CRS)
transactions_gdf = transactions_gdf.to_crs(gdf_2023.crs)

In [64]:
# Spatial join: for each transaction, find the buurt it lies in
joined = gpd.sjoin(transactions_gdf, gdf_2023[["statcode", "geometry"]], how="left", predicate="within")

# BU_CODE is the buurtcode
transactions_with_buurtcode = joined.drop(columns=["index_right"])


In [65]:
transactions_with_buurtcode.head()

,SELLING_PRICE,DATE_OF_SALE,HOUSETYPE,SOC_CODE,LIVING_AREA,PLOT_AREA,CONSTRUCTION_YEAR,LAT,LON,DESTINATION,STORAGE_OR_SHED,GARAGE,MONUMENT,geometry,statcode
0,410399.0,2023-11-23,0,22,69.0,50.0,2006,51.930438,4.597374,0,1,1,0,POINT (100677.483 438291.535),BU05020338
1,553211.0,2024-08-31,3,10,143.0,170.0,1964,51.914836,4.439337,0,1,0,0,POINT (89785.833 436685.755),BU05990322
2,281874.0,2024-03-15,0,22,103.0,53.0,1949,51.910093,4.372199,0,1,0,0,POINT (85159.5 436220.474),BU06060310
3,311783.0,2017-10-30,2,9,182.0,164.0,1969,51.918368,4.322173,0,0,0,0,POINT (81730.962 437190.304),BU06220801
4,864350.0,2023-08-03,3,10,148.0,322.0,1938,51.932097,4.464516,0,0,0,0,POINT (91542.489 438583.754),BU05990531


In [66]:
transactions_with_buurtcode.drop(columns=["geometry"], inplace=True)
transactions_with_buurtcode.rename(columns={"statcode": "BUURTCODE"}, inplace=True)
transactions_with_buurtcode

,SELLING_PRICE,DATE_OF_SALE,HOUSETYPE,SOC_CODE,LIVING_AREA,PLOT_AREA,CONSTRUCTION_YEAR,LAT,LON,DESTINATION,STORAGE_OR_SHED,GARAGE,MONUMENT,BUURTCODE
0,410399.0,2023-11-23,0,22,69.0,50.0,2006,51.930438,4.597374,0,1,1,0,BU05020338
1,553211.0,2024-08-31,3,10,143.0,170.0,1964,51.914836,4.439337,0,1,0,0,BU05990322
2,281874.0,2024-03-15,0,22,103.0,53.0,1949,51.910093,4.372199,0,1,0,0,BU06060310
3,311783.0,2017-10-30,2,9,182.0,164.0,1969,51.918368,4.322173,0,0,0,0,BU06220801
4,864350.0,2023-08-03,3,10,148.0,322.0,1938,51.932097,4.464516,0,0,0,0,BU05990531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,219846.0,2019-08-19,0,22,88.0,53.0,1951,51.910131,4.404810,0,1,0,0,BU06060004
199996,369542.0,2023-09-30,0,24,94.0,50.0,1970,51.921255,4.562029,0,1,0,0,BU05020994
199997,271956.0,2018-12-23,0,25,83.0,52.0,1924,51.907061,4.498734,0,1,0,0,BU05991079
199998,140154.0,2018-01-27,3,10,89.0,201.0,1980,51.936764,4.371009,0,1,0,0,BU06060706


In [67]:
df_transaction_rott = pd.read_csv("../../housing-data/rotterdam_transaction_data.csv")
df_transaction_rott

,TRANSID,DATUM,WONINGTYPE,SOC,CALCOPP,KAVOPP,BOUWJAAR,BUURTCODE,LAT,LON,BESTEMMING,DUMMY_BERGING,DUMMY_GARAGE,DUMMY_MONUMENT,FOTOWIJZER_TYPE,LOG_KOOPSOM
0,7537440,2021-11-29,3.0,1131,97,132,1976,11,51.849001,4.541944,1.0,1,0,0,6,12.608199
1,7308578,2021-09-10,2.0,1121,181,457,1976,11,51.848052,4.544069,1.0,1,1,0,2,13.270783
2,4689877,2016-06-15,3.0,1131,90,109,2000,209,51.949338,4.512931,1.0,1,0,0,6,12.301383
3,5421309,2019-02-01,4.0,1111,196,346,1993,346,51.856716,4.403271,1.0,1,0,0,1,13.011432
4,6215385,2020-10-29,3.0,1131,87,111,1900,382,51.908115,4.350812,1.0,1,0,0,6,12.528156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108994,5490122,2019-05-08,0.0,1183,85,0,1968,82,51.932054,4.596247,1.0,1,0,0,9,12.107357
108995,5022491,2017-08-31,4.0,1111,232,756,1992,582,51.847523,4.156492,1.0,1,1,0,1,13.226904
108996,5014000,2017-08-01,3.0,1171,111,104,1980,239,51.944781,4.566412,1.0,1,0,0,4,12.122691
108997,4641789,2016-03-02,2.0,1121,100,104,1935,78,51.923281,4.586171,1.0,1,0,0,2,12.100712


In [68]:
#convert selling price column in transactions_with_buurtcode to log scale
transactions_with_buurtcode["LOG_SELLING_PRICE"] = np.log(transactions_with_buurtcode["SELLING_PRICE"])
transactions_with_buurtcode.drop(columns=["SELLING_PRICE"], inplace=True)

In [69]:
transactions_with_buurtcode = transactions_with_buurtcode.reset_index(names='TRANSID')
transactions_with_buurtcode.rename(columns={"DATE_OF_SALE": "DATUM","HOUSETYPE": "WONINGTYPE","SOC_CODE": "SOC","LIVING_AREA": "CALCOPP","PLOT_AREA": "KAVOPP","CONSTRUCTION_YEAR": "BOUWJAAR","DESTINATION": "BESTEMMING","STORAGE_OR_SHED": "DUMMY_BERGING","GARAGE": "DUMMY_GARAGE","MONUMENT": "DUMMY_MONUMENT", "LOG_SELLING_PRICE":"LOG_KOOPSOM"}, inplace=True)

In [70]:
transactions_with_buurtcode

,TRANSID,DATUM,WONINGTYPE,SOC,CALCOPP,KAVOPP,BOUWJAAR,LAT,LON,BESTEMMING,DUMMY_BERGING,DUMMY_GARAGE,DUMMY_MONUMENT,BUURTCODE,LOG_KOOPSOM
0,0,2023-11-23,0,22,69.0,50.0,2006,51.930438,4.597374,0,1,1,0,BU05020338,12.924885
1,1,2024-08-31,3,10,143.0,170.0,1964,51.914836,4.439337,0,1,0,0,BU05990322,13.223495
2,2,2024-03-15,0,22,103.0,53.0,1949,51.910093,4.372199,0,1,0,0,BU06060310,12.549215
3,3,2017-10-30,2,9,182.0,164.0,1969,51.918368,4.322173,0,0,0,0,BU06220801,12.650063
4,4,2023-08-03,3,10,148.0,322.0,1938,51.932097,4.464516,0,0,0,0,BU05990531,13.669733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,2019-08-19,0,22,88.0,53.0,1951,51.910131,4.404810,0,1,0,0,BU06060004,12.300683
199996,199996,2023-09-30,0,24,94.0,50.0,1970,51.921255,4.562029,0,1,0,0,BU05020994,12.820020
199997,199997,2018-12-23,0,25,83.0,52.0,1924,51.907061,4.498734,0,1,0,0,BU05991079,12.513396
199998,199998,2018-01-27,3,10,89.0,201.0,1980,51.936764,4.371009,0,1,0,0,BU06060706,11.850497


In [71]:
adj_rott = pd.read_csv("../../housing-data/rotterdam_adj_2023.csv", index_col=0)
adj_rott

,BU04890101,BU04890102,BU04890103,BU04890204,BU04890205,BU04890206,BU04890207,BU04890308,BU04890409,BU04890410,...,BU06060904,BU06061001,BU06061002,BU06061003,BU06061004,BU06061005,BU06061006,BU06061007,BU06061101,BU06061102
BU04890101,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BU04890102,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BU04890103,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BU04890204,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BU04890205,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BU06061005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
BU06061006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
BU06061007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
BU06061101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [72]:
count = 0

for _,row in transactions_with_buurtcode.iterrows():
    if row["BUURTCODE"] not in adj_rott.index:
        count = count+1
print(f"Number of transactions with buurtcode not in adjacency matrix: {count}")

Number of transactions with buurtcode not in adjacency matrix: 3179


In [73]:
#drop the transactions with buurtcode not in adjacency matrix
transactions_with_buurtcode = transactions_with_buurtcode[transactions_with_buurtcode["BUURTCODE"].isin(adj_rott.index)]

In [74]:
count = 0

for _,row in transactions_with_buurtcode.iterrows():
    if row["BUURTCODE"] not in adj_rott.index:
        count = count+1
print(f"Number of transactions with buurtcode not in adjacency matrix: {count}")

Number of transactions with buurtcode not in adjacency matrix: 0


In [78]:
from sklearn.preprocessing import LabelEncoder

buurten = adj_rott.columns.values
label_encoder = LabelEncoder()
label_encoder.fit(buurten)

transactions_with_buurtcode["BUURTCODE"] = label_encoder.transform(transactions_with_buurtcode["BUURTCODE"])

C:\Users\AratrikaD\AppData\Local\Temp\ipykernel_19640\3727904337.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_with_buurtcode["BUURTCODE"] = label_encoder.transform(transactions_with_buurtcode["BUURTCODE"])


In [79]:
transactions_with_buurtcode

,TRANSID,DATUM,WONINGTYPE,SOC,CALCOPP,KAVOPP,BOUWJAAR,LAT,LON,BESTEMMING,DUMMY_BERGING,DUMMY_GARAGE,DUMMY_MONUMENT,BUURTCODE,LOG_KOOPSOM
0,0,2023-11-23,0,22,69.0,50.0,2006,51.930438,4.597374,0,1,1,0,85,12.924885
1,1,2024-08-31,3,10,143.0,170.0,1964,51.914836,4.439337,0,1,0,0,186,13.223495
2,2,2024-03-15,0,22,103.0,53.0,1949,51.910093,4.372199,0,1,0,0,296,12.549215
3,3,2017-10-30,2,9,182.0,164.0,1969,51.918368,4.322173,0,0,0,0,404,12.650063
4,4,2023-08-03,3,10,148.0,322.0,1938,51.932097,4.464516,0,0,0,0,201,13.669733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,2019-08-19,0,22,88.0,53.0,1951,51.910131,4.404810,0,1,0,0,272,12.300683
199996,199996,2023-09-30,0,24,94.0,50.0,1970,51.921255,4.562029,0,1,0,0,136,12.820020
199997,199997,2018-12-23,0,25,83.0,52.0,1924,51.907061,4.498734,0,1,0,0,220,12.513396
199998,199998,2018-01-27,3,10,89.0,201.0,1980,51.936764,4.371009,0,1,0,0,323,11.850497


In [80]:
#save the processed transactions data
transactions_with_buurtcode.to_csv("../../housing-data/synthetic_transactions.csv", index=False)